In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-11-09


Found rundate 2023-11-08 10:00:02 < 2023-11-09 at commit 273b87c6; returning commit c156a1f


'2023-11-09 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-11-09 10:00:02, NJSP was reporting 509 YTD deaths
Current YTD Deaths (2024-11-09 10:00:01-05:00): 581
Previous year YTD Deaths (adjusted; 2023-11-09 10:00:02): 509.2340536939857
Projected 2024 total: 689.4438407045579
14.1% change


FAUQStats cache miss: 2023, Thu Nov 09 10:00:02 EST 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,Mulberry St,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,NaN,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,322,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,South Randolphville Road,South Randolphville Road,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,Outwater Ln,Outwater Ln,1.0,0.0,0.0,1.0,0.0,NaN,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,Washington St,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12792,09,Hudson,0906,Jersey City,NaN,County 637,1.0,0.0,0.0,1.0,0.0,637,NaN,2023-11-04 15:50:00-04:00
12797,19,Sussex,1905,Frankford Twsp,NaN,County 565 MP .8,1.0,1.0,0.0,0.0,0.0,565,NaN,2023-11-06 10:34:00-05:00
12796,15,Ocean,1514,Lakewood Twsp,NaN,County 639,1.0,1.0,0.0,0.0,0.0,639,NaN,2023-11-06 20:22:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,27,2,15,3,46
Bergen,14,4,15,2,33
Burlington,27,4,6,1,38
Camden,16,3,14,0,33
Cape May,7,0,0,0,7
Cumberland,17,6,6,0,24
Essex,17,5,22,0,43
Gloucester,16,5,4,0,22
Hudson,8,2,4,1,14


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

85.6% through the year, 14.4% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,16,18,27
Bergen,driver,18,20,14
Burlington,driver,21,26,27
Camden,driver,17,18,16
Cape May,driver,4,4,7
...,...,...,...,...
Salem,crashes,9,11,12
Somerset,crashes,17,21,18
Sussex,crashes,6,6,11


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,56,6,30,2,19
Bergen,40,2,16,4,21
Burlington,49,2,33,5,6
Camden,41,0,17,4,21
Cape May,10,0,7,0,0
Cumberland,30,0,20,7,8
Essex,57,0,20,8,31
Gloucester,25,0,17,7,7
Hudson,14,1,9,2,4


In [9]:
projected.to_csv(PROJECTED_CSV)